# Head control

Reachy 2's head is mounted on an Orbita3D actuator, referred to as the **neck** actuator, giving 3 degrees of freedom to control the head orientation.  
> Note : the antennas are not motorized for the moment

Before starting to control the head, connect to your Reachy and turn it on. As in the other pages:

## Initialize your robot

First connect to your robot:

In [3]:
from reachy2_sdk import ReachySDK

reachy = ReachySDK(host='localhost')  # Replace with the actual IP

Cameras not initialized.


Let's check what contains the head part:

In [5]:
reachy.head.joints

{'neck.roll': <OrbitaJoint axis_type="roll" present_position=20.0 goal_position=20.0 >,
'neck.pitch': <OrbitaJoint axis_type="pitch" present_position=30.0 goal_position=30.0 >,
'neck.yaw': <OrbitaJoint axis_type="yaw" present_position=-10.0 goal_position=-10.0 >}

The head is composed of **three joints**:
- neck.roll
- neck.pitch
- neck.yaw

All joints belong to the same Orbita3D actuator, referred to as the **neck**.

To start, we need to turn the head on:

In [6]:
reachy.head.turn_on()

You could of course turn on the whole robot by calling `reachy.turn_on()` directly.

## Head moves

There are several ways to control the head movements:
- using the `look_at()`, `rotate_to()` and `orient()` methods, called directly at the **head** level. These methods works as in the moves_introduction example
- controlling the joints goal positions, namely **reachy.head.neck.roll**, **reachy.head.neck.pitch** and **reachy.head.neck.yaw**.

## Head moves methods

### look_at()

You can use the `look_at()` function to make the head look at a specific point in space. This point must be given in Reachy 2's coordinate system in **meters**. The coordinate system is the one we have seen previously:

* the X axis corresponds to the foward arrow,
* the Y axis corresponds to the right to left arrow,
* the Z axis corresponds to the up arrow.

The origin of this coordinate system is located in the upper part of the robot trunk.

If you want Reachy to look forward you can send it the following.

In [7]:
reachy.head.look_at(x=0.5, y=0, z=0.2, duration=1.0)

id: 24

You can use multiple *look_at* to chain head movements, or even chain them with the `rotate_to()` and `orient()` functions described below. As seen in the moves_introduction, the commands on the head will be stacked.
For example:

In [8]:
import time

look_right = reachy.head.look_at(x=0.5, y=-0.5, z=0.1, duration=1.0)
look_down = reachy.head.look_at(x=0.5, y=0, z=-0.4, duration=1.0)
look_left = reachy.head.look_at(x=0.5, y=0.3, z=-0.3, duration=1.0)
look_front = reachy.head.look_at(x=0.5, y=0, z=0, duration=1.0)

The best way to understand how to use the *look_at* is to play with it. Picture a position you would like Reachy's head to be in, guess a point which could match for the *look_at* and check if you got it right!

#### End effector followed by head

Another cool thing is that we can combine Reachy's kinematics with the *look_at* so that Reachy's head follows its hand!

In [9]:
x, y, z = reachy.r_arm.forward_kinematics()[:3, -1]
reachy.head.look_at(x=x, y=y, z=z, duration=1.0)

time.sleep(0.5)

while True:
    x, y, z = reachy.r_arm.forward_kinematics()[:3, -1]
    reachy.head.look_at(x=x, y=y, z=z, duration=0.1)

KeyboardInterrupt: 

What the code says is that we compute the forward_kinematics, and the x, y, z of Reachy's right end-effector in the Reachy's coordinates system will be the coordinates of the point used by the *look_at*.

> **Manually stop the previous execution cell**, because of the `while True` loop

### rotate_to()

The `rotate_to()` function is another way to control the head. You directly control the joint of the neck, giving the roll, pitch and yaw angles in degrees. The rotation is made in the order: roll, pitch, yaw, in the Orbita3D coordinate system.

In [10]:
reachy.head.rotate_to(roll=0, pitch=-10, yaw=0, duration=1.0)

id: 404

### orient()

The last method to control the head is the `orient()` method. You can control the head with a quaternion.

You can use [pyquaternion library](https://kieranwynn.github.io/pyquaternion/) to create suitable quaternion for this method.

In [11]:
from pyquaternion import Quaternion

q = Quaternion(axis=[1, 0, 0], angle=3.14159265)
reachy.head.turn_on()
reachy.head.orient(q)

id: 405

## Read head position

You can read the head orientation in two different ways:

- using the `get_orientation()` method, which returns a quaternion
- using the `get_joints_positions()` method, which the neck's roll, pitch and yaw present_position.

### get_orientation()

In [12]:
q = reachy.head.get_orientation()
print(q)

-0.000 +1.000i +0.000j +0.000k


### get_joints_positions()

In case you feel more comfortable using roll, pitch, yaw angles rather than working with quaternions, you can retrieve those values from the **neck joints**.

In [15]:
reachy.head.rotate_to(20, 30, -10)

time.sleep(2)

reachy.head.get_joints_positions()

[20.0, 30.0, -10.0]

Be careful that contrary to the quaternion that offers a unique representation of a rotation, it is not the case of the euler angles. Several angles combination can lead to the same orientation in space. For example:


In [18]:
reachy.head.rotate_to(70, -100, 80)  #  roll=70, pitch=-100, yaw=80

time.sleep(2)

reachy.head.get_joints_positions()

[-110.0, -80.0, -100.0]

The values are different, nevertheless it is the same final orientation. You can convince yourself doing:

In [17]:
reachy.head.rotate_to(-110, -80, -100)

id: 410

The head won't move.